# Признаки активности для дня недели, недели года, дня месяца

Импортируем библиотеки и делаем некоторые настройки

In [1]:
import pandas as pd
import numpy as np

In [2]:
import datetime
from datetime import datetime, date

In [3]:
import warnings
warnings.filterwarnings('ignore')

Загружаем данные

In [4]:
events_dates = pd.read_csv('DATA\\events_dates.csv')

In [5]:
#сразу переименуем столбцы
events_dates = events_dates.rename(columns={'date_1':'Edate_1',
                                            'date_2':'Edate_2',
                                            'date_3':'Edate_3'})   

In [6]:
submissions_dates = pd.read_csv('DATA\\submissions_dates.csv')

In [7]:
#сразу переименуем столбцы
submissions_dates = submissions_dates.rename(columns={'date_1':'Sdate_1',
                                                      'date_2':'Sdate_2',
                                                      'date_3':'Sdate_3'})

Объединяем данные EVENTS и SUBMISSIONS

In [8]:
dates = events_dates.merge(submissions_dates, on=['user_id'], how='outer')

In [9]:
dates = dates.fillna(0)#это временно

In [10]:
# В тех ячейках, где отсутствуют сабмиты, заполню датами из  Edate_1
dates['Sdate_1'][dates.Sdate_1 == 0] = dates.Edate_1

In [11]:
del submissions_dates
del events_dates

Работаем с EVENTS-датами

In [12]:
#преобразуем даты EVENTS в формат datatime
f = lambda x: datetime.strptime(x, '%Y-%m-%d') 
dates.Edate_1 = dates.Edate_1.apply(f)
dates.Edate_2 = dates.Edate_2.apply(f)
dates.Edate_3 = dates.Edate_3.apply(f)

In [13]:
d_1 = pd.DateOffset(days = 1)# переменные, равные 1 и 2 дням соответственно
d_2 = pd.DateOffset(days = 2)

In [14]:
dates.Edate_2 = dates.Edate_1 + d_1# заполняем датами 2й и 3й дни
dates.Edate_3 = dates.Edate_1 + d_2

In [15]:
dates['week_1'] = dates.Edate_1.dt.week# номер недели 1, 2 и 3 дней
dates['week_2'] = dates.Edate_2.dt.week
dates['week_3'] = dates.Edate_3.dt.week
dates['weekday_1'] = dates.Edate_1.dt.weekday# день недели для 1, 2 
dates['weekday_2'] = dates.Edate_2.dt.weekday# и 3 дней
dates['weekday_3'] = dates.Edate_3.dt.weekday
dates['day_1'] = dates.Edate_1.dt.day# день месяца для 1, 2 и 3 дней
dates['day_2'] = dates.Edate_2.dt.day
dates['day_3'] = dates.Edate_3.dt.day

Загружаем параметры активности по неделям, по дням недели и по дням месяца

In [16]:
week = pd.read_csv('DATA\koef_week.csv')
weekday = pd.read_csv('DATA\koef_weekday.csv')
day = pd.read_csv('DATA\koef_day.csv')

Выполняем слияние, переименование и удаление ненужных колонок

In [17]:
dates = dates.merge(week, left_on='week_1', right_on='week', how='inner')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Ewk_1'})

In [18]:
dates = dates.merge(week, left_on='week_2', right_on='week', how='inner')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Ewk_2'})

In [19]:
dates = dates.merge(week, left_on='week_3', right_on='week', how='inner')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Ewk_3'})

In [20]:
dates = dates.merge(weekday, left_on='weekday_1', right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Ewd_1'})

In [21]:
dates = dates.merge(weekday, left_on='weekday_2', right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Ewd_2'})

In [22]:
dates = dates.merge(weekday, left_on='weekday_3', right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Ewd_3'})

In [23]:
dates = dates.merge(day, left_on='day_1', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Eday_1'})

In [24]:
dates = dates.merge(day, left_on='day_2', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Eday_2'})

In [25]:
dates = dates.merge(day, left_on='day_3', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Eday_3'})

In [26]:
dates = dates.drop(['week_1', 'week_2', 'week_3',
                    'weekday_1', 'weekday_2', 'weekday_3',
                    'day_1', 'day_2', 'day_3'], axis=1)

Теперь c SUBMISSION

In [27]:
dates.Sdate_1 = dates.Sdate_1.apply(f)#преобразуем даты в формат datatime

In [28]:
dates.Sdate_2 = dates.Sdate_1 + d_1# заполняем датами 2й и 3й дни
dates.Sdate_3 = dates.Sdate_1 + d_2

In [29]:
dates['week_1'] = dates.Sdate_1.dt.week# номер недели 1, 2 и 3 дней
dates['week_2'] = dates.Sdate_2.dt.week
dates['week_3'] = dates.Sdate_3.dt.week
dates['weekday_1'] = dates.Sdate_1.dt.weekday# день недели 
dates['weekday_2'] = dates.Sdate_2.dt.weekday# для 1, 2 и 3 дней
dates['weekday_3'] = dates.Sdate_3.dt.weekday
dates['day_1'] = dates.Sdate_1.dt.day# день месяца для 1, 2 и 3 дней
dates['day_2'] = dates.Sdate_2.dt.day
dates['day_3'] = dates.Sdate_3.dt.day

Выполняем слияние, переименование и удаление ненужных колонок

In [30]:
dates = dates.merge(week,left_on='week_1',right_on='week')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Swk_1'})

In [31]:
dates = dates.merge(week,left_on='week_2',right_on='week')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Swk_2'})

In [32]:
dates = dates.merge(week,left_on='week_3',right_on='week')
dates = dates.drop('week', axis=1).rename(columns={'koef_wk': 'Swk_3'})

In [33]:
dates = dates.merge(weekday,left_on='weekday_1',right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Swd_1'})

In [34]:
dates = dates.merge(weekday,left_on='weekday_2',right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Swd_2'})

In [35]:
dates = dates.merge(weekday,left_on='weekday_3',right_on='weekday')
dates = dates.drop('weekday', axis=1).rename(columns={'koef_wd': 'Swd_3'})

In [36]:
dates = dates.merge(day, left_on='day_1', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Sday_1'})

In [37]:
dates = dates.merge(day, left_on='day_2', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Sday_2'})

In [38]:
dates = dates.merge(day, left_on='day_3', right_on='day')
dates = dates.drop('day', axis=1).rename(columns={'koef_dm': 'Sday_3'})

In [39]:
dates.sort_values(by=['user_id'], axis=0, inplace=True)

In [40]:
dates['koef_week_mean']=(dates.Swk_1 + dates.Swk_2 + dates.Swk_3)/3

In [41]:
dates = dates.drop(['Edate_1', 'Edate_2', 'Edate_3', 
                    'Sdate_1', 'Sdate_2', 'Sdate_3',
                    'week_1', 'week_2', 'week_3',
                    'weekday_1', 'weekday_2', 'weekday_3',
                    'day_1', 'day_2', 'day_3',
                    'Ewk_1', 'Ewk_2', 'Ewk_3',
                    'Ewd_1', 'Ewd_2', 'Ewd_3',
                    'Eday_1', 'Eday_2', 'Eday_3',
                    'Swk_1', 'Swk_2', 'Swk_3'], axis=1)

In [42]:
dates = dates.rename(columns={'Swd_1': 'koef_weekday_1',
                              'Swd_2': 'koef_weekday_2',
                              'Swd_3': 'koef_weekday_3',
                              'Sday_1': 'koef_day_1',
                              'Sday_2': 'koef_day_2',
                              'Sday_3': 'koef_day_3'})

Сохраним результаты в файл:

In [43]:
dates.to_csv('DATA\\week_weekday_day.zip', index=False, compression='zip')

In [44]:
dates

,user_id,koef_weekday_1,koef_weekday_2,koef_weekday_3,koef_day_1,koef_day_2,koef_day_3,koef_week_mean
0,1,0.140223,0.118389,0.130860,0.034963,0.035346,0.032876,0.018919
11212,2,0.161449,0.151801,0.140223,0.031171,0.031415,0.032599,0.019003
9964,3,0.145941,0.151336,0.161449,0.028597,0.031867,0.031554,0.019739
19601,4,0.145941,0.151336,0.161449,0.033050,0.032876,0.032771,0.019739
14483,5,0.161449,0.151801,0.140223,0.032945,0.033537,0.032632,0.015910
...,...,...,...,...,...,...,...,...
14958,26796,0.118389,0.130860,0.145941,0.032945,0.033537,0.032632,0.014664
15158,26797,0.151801,0.140223,0.118389,0.032876,0.032771,0.032528,0.024070
22192,26798,0.151336,0.161449,0.151801,0.033537,0.032632,0.028597,0.019831
4724,26799,0.140223,0.118389,0.130860,0.031902,0.030232,0.031275,0.026942


In [45]:
dates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25418 entries, 0 to 20886
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   user_id         25418 non-null  int64  
 1   koef_weekday_1  25418 non-null  float64
 2   koef_weekday_2  25418 non-null  float64
 3   koef_weekday_3  25418 non-null  float64
 4   koef_day_1      25418 non-null  float64
 5   koef_day_2      25418 non-null  float64
 6   koef_day_3      25418 non-null  float64
 7   koef_week_mean  25418 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 1.7 MB
